# Analysis of Spotify user data from first half Oct 2015

We analyze user and song data from the first half of October, 2015 in order to draw conclusions and form insights into product usage. We are interested mainly in three questions:

- Are listening habits between men and women significantly different?
- How do free and premium users use Spotify's product differently?
- What are the best predictors of how heavily a user uses Spotify?

In [1]:
import numpy as np
from scipy import stats
import sqlite3
import csv

Parse the datasets into SQLite3.

In [2]:
conn = sqlite3.connect('data.db')
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='users';")
if len(c.fetchall()) == 0:
    c.execute("""CREATE TABLE users
                    (gender text, age_range text, country text, acct_age_weeks integer, user_id text)""")
    header = True
    with open('user_data_sample.csv') as userfile:
        for row in csv.reader(userfile):
            if header:
                header = False
            else:
                c.execute("INSERT INTO users VALUES (?,?,?,?,?)", row)
    c.execute("""CREATE TABLE songs
                    (ms_played integer, context text, track_id text, product text, end_timestamp real, user_id text)""")

    header = True
    with open('end_song_sample.csv') as songfile:
        for row in csv.reader(songfile):
            if header:
                header = False
            else:
                c.execute("INSERT INTO songs VALUES (?,?,?,?,?,?)", row)
    conn.commit()

Quick sanity check, and then grab male/female uids.

In [3]:
c.execute("select user_id from users where gender = 'male'")
male_uids = set(map(lambda l: l[0], c.fetchall()))
num_males = len(male_uids)

c.execute("select user_id from users where gender = 'female'")
female_uids = set(map(lambda l: l[0], c.fetchall()))
num_females = len(female_uids)

c.execute("select user_id from users where gender != 'male' and gender != 'female'")
num_others = len(c.fetchall())

c.execute("select user_id from users")

assert(len(c.fetchall()) == num_males + num_females + num_others)

First, we want to determine whether male and female listeners are significantly different in their overall listening (in terms of the count of track listens and total time spent listening).

After calculating the total time played/total songs listened for a user, we append them to either the male or female samples list. We ignore listeners for which gender is unknown. Then we use a 2 independent samples Welch's T Test on the total listens and total playtime.

In [4]:
try:
    male_listens = np.load('male_listens.npy')
    male_playtime = np.load('male_playtime.npy')
    female_listens = np.load('female_listens.npy')
    female_playtime = np.load('female_playtime.npy')
except:
    male_listens = []
    male_playtime = []
    female_listens = []
    female_playtime = []
    c.execute("select user_id, count(*), sum(ms_played) from songs group by user_id")
    for uid, listens, playtime in c.fetchall():
        if uid in male_uids:
            male_listens.append(listens)
            male_playtime.append(playtime)
        else:
            female_listens.append(listens)
            female_playtime.append(playtime)

    np.save('male_listens.npy', male_listens)
    np.save('male_playtime.npy', male_playtime)
    np.save('female_listens.npy', female_listens)
    np.save('female_playtime.npy', female_playtime)

In [5]:
print(stats.ttest_ind(female_playtime, male_playtime, equal_var=False))

Ttest_indResult(statistic=0.46218877630132438, pvalue=0.64395648282023876)


We have a p-value of greater than the standard threshold of .05, so we cannot reject the null hypothesis that male and female listening habits measured in playtime have the same underlying distribution.

In [6]:
print(stats.ttest_ind(female_listens, male_listens, equal_var=False))

Ttest_indResult(statistic=0.61478753599674751, pvalue=0.53870973819945345)


We again have a p-value of greater than the standard threshold of .05, so we again cannot reject the null hypothesis that male and female listening habits measured in playtime have the same underlying distribution. The two above tests indicate to us that, measured in broad strokes of sheer quantity, males and females listen to approximately the same amount of music.

## How do free and premium users use Spotify's product differently?

Now we would like to perform another analysis, that of whether (and if so, how) free and premium users use Spotify. We define free song plays to be song plays on products 'free' and 'open', and premium song plays to be song plays on products 'premium' and 'basic-desktop' (since an internet search indicates that [basic-desktop allows for unlimited streaming only with ads](https://community.spotify.com/t5/Accounts-and-Subscriptions/Paying-for-quot-basic-desktop-London-quot-What-is-this/td-p/242528), which we assume does not fundamentally affect the product as much as shuffle-only, limited skips does).

In our dataset, some users played songs as both free and premium users, in which case we divide up their plays into two users. We assume that underlying motivations for a user on the product change significantly after a user purchases a premium subscription, so much so that their behaviors before and after subscribing constitute two users.

In [7]:
free = {}
premium = {}
c.execute("select user_id, product, context, count(*) from songs group by user_id, product, context")
free_user_dict = {}
premium_user_dict = {}
plays = c.fetchall()
for p in plays:
    if p[1] == 'premium' or p[1] == 'basic-desktop':
        if p[0] not in premium_user_dict:
            premium_user_dict[p[0]] = {'album': 0, 'collection': 0, 'playlist': 0, 'artist': 0, 'app': 0, 'me': 0, 'unknown': 0, 'search': 0}
        premium_user_dict[p[0]][p[2]] = p[3]
    else:
        if p[0] not in free_user_dict:    
            free_user_dict[p[0]] = {'album': 0, 'collection': 0, 'playlist': 0, 'artist': 0, 'app': 0, 'me': 0, 'unknown': 0, 'search': 0}
        free_user_dict[p[0]][p[2]] = p[3]
        
free['album'] = [user['album'] for user in free_user_dict.values()]
free['collection'] = [user['collection'] for user in free_user_dict.values()]
free['playlist'] = [user['playlist'] for user in free_user_dict.values()]
free['artist'] = [user['artist'] for user in free_user_dict.values()]
free['app'] = [user['app'] for user in free_user_dict.values()]
free['me'] = [user['me'] for user in free_user_dict.values()]
free['search'] = [user['search'] for user in free_user_dict.values()]

premium['album'] = [user['album'] for user in premium_user_dict.values()]
premium['collection'] = [user['collection'] for user in premium_user_dict.values()]
premium['playlist'] = [user['playlist'] for user in premium_user_dict.values()]
premium['artist'] = [user['artist'] for user in premium_user_dict.values()]
premium['app'] = [user['app'] for user in premium_user_dict.values()]
premium['me'] = [user['me'] for user in premium_user_dict.values()]
premium['search'] = [user['search'] for user in premium_user_dict.values()]

In [8]:
print("t-test on album plays: pvalue = " + str(stats.ttest_ind(free['album'], premium['album'], equal_var=False)[1]))
print("t-test on collection plays: pvalue = " + str(stats.ttest_ind(free['collection'], premium['collection'], equal_var=False)[1]))
print("t-test on playlist plays: pvalue = " + str(stats.ttest_ind(free['playlist'], premium['playlist'], equal_var=False)[1]))
print("t-test on artist plays: pvalue = " + str(stats.ttest_ind(free['artist'], premium['artist'], equal_var=False)[1]))
print("t-test on app plays: pvalue = " + str(stats.ttest_ind(free['app'], premium['app'], equal_var=False)[1]))
print("t-test on me plays: pvalue = " + str(stats.ttest_ind(free['me'], premium['me'], equal_var=False)[1]))
print("t-test on search plays: pvalue = " + str(stats.ttest_ind(free['search'], premium['search'], equal_var=False)[1]))

t-test on album plays: pvalue = 1.99208935034e-15
t-test on collection plays: pvalue = 9.74644034405e-08
t-test on playlist plays: pvalue = 6.75027572797e-21
t-test on artist plays: pvalue = 3.48403948155e-05
t-test on app plays: pvalue = 0.00755635933161
t-test on me plays: pvalue = 1.68107719263e-05
t-test on search plays: pvalue = 6.87765889628e-23


Here we conclude that free and premium users play songs from every single context at statistically significant differences.  We then explore which contexts premium users prefer, and vice versa.

In [9]:
f = stats.describe(free['album'])
p = stats.describe(premium['album'])
print("album stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

f = stats.describe(free['collection'])
p = stats.describe(premium['collection'])
print("collection stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

f = stats.describe(free['playlist'])
p = stats.describe(premium['playlist'])
print("playlist stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

f = stats.describe(free['artist'])
p = stats.describe(premium['artist'])
print("artist stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

f = stats.describe(free['app'])
p = stats.describe(premium['app'])
print("app stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

f = stats.describe(free['me'])
p = stats.describe(premium['me'])
print("me stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

f = stats.describe(free['search'])
p = stats.describe(premium['search'])
print("search stats:\nfree mean: {0}\npremium mean: {1}\nfree sd: {2}\npremium sd: {3}\n".format(f[2], p[2], np.sqrt(f[3]), np.sqrt(p[3])))

album stats:
free mean: 15.209075384240059
premium mean: 28.33735632183908
free sd: 43.90034080172029
premium sd: 65.36755447158622

collection stats:
free mean: 12.816784581605269
premium mean: 32.47471264367816
free sd: 74.42724476139055
premium sd: 149.33240722568718

playlist stats:
free mean: 58.25579409612101
premium mean: 112.17931034482758
free sd: 143.53674340072374
premium sd: 227.85205501299995

artist stats:
free mean: 18.05550134179068
premium mean: 29.30574712643678
free sd: 57.5989590301574
premium sd: 109.95287647252512

app stats:
free mean: 3.2683581361307636
premium mean: 5.275287356321839
free sd: 21.780279606482363
premium sd: 29.658155277140356

me stats:
free mean: 1.3045864845084167
premium mean: 0.167816091954023
free sd: 22.69175523103474
premium sd: 3.4633543407955907

search stats:
free mean: 2.3273969260795315
premium mean: 7.547701149425287
free sd: 16.811578306028686
premium sd: 20.451643422755676



We see here some interesting results from our test. In every context except 'me', premium users play nearly twice as many songs. Since every play command must be either explicitly or implicitly issued, ie song plays are conscious decisions by users, we look at the largest absolute increases, that of songs played from playlists and collections. 

The fact that these categories increased the most suggest that they are driving factors in why users subscribe, which confirms the design philosophy behind the free version of Spotify- to reduce the functionality and accessiblity of the best parts of the premium product (the feeling of "owning" every song and album that Spotify serves and the power and privilege of playing any song at will) to the point where users feel frustrated and powerless, which they can remedy by purchasing a subscription.

Also interesting is the trend seen in songs played from the 'me' context. Whereas free user usage of the 'me' context vary wildly (evidence by a large standard deviation), premium usage of the 'me' context plummets to virtual disuse. This pattern of behavior suggests that free users are using the 'me' context to emulate the premium product. Whereas a premium user can explicitly select single songs to play, if a free user wishes to listen to a particular song, they must play the album containing the song, the artist page containing the song, or a playlist containing the song. 

However, the cost of waiting on an album or artist page for a specfic song to play is very high. Often the random shuffling imposed on free users will lead the user on a wild goose chase through skits, intros, or unwanted remixes, whereas a free user can pack a playlist with only songs the free user truly enjoys in order to maximize the average utility when the client shuffles to the next random song.

Playlists in essence return some control over song choice to the user, and the 'me' context, which can and often is used as a pseudo-playlist, scales poorly and so is disregarded by the premium users. In conclusion, we find that playlists are a very important driver of premium subscriptions, which makes a case for focusing engineering efforts to ensure that playlist usability and functionality is always optimal.

## What are the best predictors of how heavily a user uses Spotify?

We wish to predict how heavily a user will user Spotify based on demographic data. As the bandwidth cost of streaming an mp3 is low, and is mitigated to a certain extent by aggressive predictive caching on the part of Spotify, we measure instead the number of songs played in the first half of October by a user, since licensing costs generally outweight server costs for streaming services.

We now explore how to predict how many songs a user will play by running a linear regression on features: gender, age (value set to middle of age bucket, with 55+ set arbitrarily to 60), acct_age (log of, to emphasize difference between young accounts), premium status, and nationality (one-hot encoded).

In [10]:
A = None
b = None
countries = {'IS': 41, 'CW': 50, 'IL': 65, 'CZ': 32, 'NI': 42, 'EC': 39, 'SI': 68, 'GB': 17, 'MW': 49, 'CH': 30, 'A1': 64, 'IE': 12, 'ES': 4, 'AU': 22, 'NL': 6, 'RO': 52, 'ZZ': 60, 'SK': 58, 'IT': 7, 'ZW': 54, 'PA': 62, 'UY': 33, 'GT': 38, 'PE': 10, 'PT': 44, 'PH': 11, 'AT': 43, 'CR': 36, 'AR': 3, 'NO': 24, 'FR': 0, 'BR': 13, 'IN': 61, 'LV': 25, 'HN': 45, 'RU': 63, 'GR': 55, 'HK': 27, 'AP': 57, 'TW': 37, 'EE': 21, 'MX': 8, 'FI': 18, 'HU': 31, 'MT': 47, 'JP': 46, 'CY': 66, 'MC': 53, 'DK': 23, 'DE': 2, 'LT': 35, 'US': 1, 'PY': 48, 'SG': 29, 'CL': 15, 'PL': 19, 'NZ': 34, 'MY': 20, 'BO': 51, 'SE': 5, 'SV': 56, 'BE': 14, 'PR': 67, 'DO': 40, 'LI': 59, 'CA': 9, 'BG': 26, 'TR': 16, 'CO': 28}
rev_countries = {0: 'FR', 1: 'US', 2: 'DE', 3: 'AR', 4: 'ES', 5: 'SE', 6: 'NL', 7: 'IT', 8: 'MX', 9: 'CA', 10: 'PE', 11: 'PH', 12: 'IE', 13: 'BR', 14: 'BE', 15: 'CL', 16: 'TR', 17: 'GB', 18: 'FI', 19: 'PL', 20: 'MY', 21: 'EE', 22: 'AU', 23: 'DK', 24: 'NO', 25: 'LV', 26: 'BG', 27: 'HK', 28: 'CO', 29: 'SG', 30: 'CH', 31: 'HU', 32: 'CZ', 33: 'UY', 34: 'NZ', 35: 'LT', 36: 'CR', 37: 'TW', 38: 'GT', 39: 'EC', 40: 'DO', 41: 'IS', 42: 'NI', 43: 'AT', 44: 'PT', 45: 'HN', 46: 'JP', 47: 'MT', 48: 'PY', 49: 'MW', 50: 'CW', 51: 'BO', 52: 'RO', 53: 'MC', 54: 'ZW', 55: 'GR', 56: 'SV', 57: 'AP', 58: 'SK', 59: 'LI', 60: 'ZZ', 61: 'IN', 62: 'PA', 63: 'RU', 64: 'A1', 65: 'IL', 66: 'CY', 67: 'PR', 68: 'SI'}
age_translate = {'0 - 17': 8.5, '18 - 24': 21, '25 - 29': 27, '30 - 34': 32, '35 - 44': 39.5, '45 - 54': 49.5, '55+': 60, '': -1}

c.execute("select user_id, product, count(*) from songs group by user_id, product")

for u in c.fetchall():
    c.execute("select gender, age_range, acct_age_weeks, country from users where user_id = (?)", (u[0],))
    user = c.fetchone()
    g = 1 if user[0] == 'male' else 0 if user[0] == 'female' else -1
    ar = age_translate[user[1]]
    ac = np.log(int(user[2])) if user[2] > 0 else np.log(.5)
    p = 1 if u[1] == 'premium' else 0
    # create the one-hot encoding of country
    if g == -1 or ar == -1 or user[3] == 'ZZ':
        pass
    ct = np.zeros((1, 69))
    ct[0][countries[user[3]]] = 1
    
    if A is None or b is None:
        A = np.append(np.array([[g, ar, ac, p]]), ct, axis=1)
        b = np.array([[u[2]]])
    else:
        A = np.append(A, np.append([[g, ar, ac, p]], ct, axis=1), axis=0)
        b = np.append(b, [[u[2]]], axis=0)

In [11]:
x = np.linalg.lstsq(A, b)[0]
print("coefficient of gender: " + str(x[0][0]))
print("coefficient of age: " + str(x[1][0]))
print("coefficient of account age: " + str(x[2][0]))
print("coefficient of premium: " + str(x[3][0]))

coefficient of gender: 0.496012898805
coefficient of age: -1.86088666294
coefficient of account age: 11.9649195597
coefficient of premium: 117.824955527


In [12]:
tmp = np.array(x[4:])
tmp.shape = (1, 69)
sc = tmp[0].argsort()
print("1st most positively correlated country: " + rev_countries[sc[-1]])
print("\t with value: " + str(tmp[0][sc[-1]]))
print("2nd most positively correlated country: " + rev_countries[sc[-2]])
print("\t with value: " + str(tmp[0][sc[-2]]))
print("3rd most positively correlated country: " + rev_countries[sc[-3]])
print("\t with value: " + str(tmp[0][sc[-3]]))
print("4th most positively correlated country: " + rev_countries[sc[-4]])
print("\t with value: " + str(tmp[0][sc[-4]]))
print("5th most positively correlated country: " + rev_countries[sc[-5]])
print("\t with value: " + str(tmp[0][sc[-5]]))

print("1st most negatively correlated country: " + rev_countries[sc[0]])
print("\t with value: " + str(tmp[0][sc[0]]))
print("2nd most negatively correlated country: " + rev_countries[sc[1]])
print("\t with value: " + str(tmp[0][sc[1]]))
print("3rd most negatively correlated country: " + rev_countries[sc[2]])
print("\t with value: " + str(tmp[0][sc[2]]))
print("4th most negatively correlated country: " + rev_countries[sc[3]])
print("\t with value: " + str(tmp[0][sc[3]]))
print("5th most negatively correlated country: " + rev_countries[sc[4]])
print("\t with value: " + str(tmp[0][sc[4]]))

1st most positively correlated country: CW
	 with value: 539.024468113
2nd most positively correlated country: RU
	 with value: 488.066670949
3rd most positively correlated country: ZW
	 with value: 367.574125967
4th most positively correlated country: PR
	 with value: 342.734444092
5th most positively correlated country: NI
	 with value: 335.422422532
1st most negatively correlated country: ZZ
	 with value: -13.6531962312
2nd most negatively correlated country: IN
	 with value: -6.05240272545
3rd most negatively correlated country: IS
	 with value: 13.7390211364
4th most negatively correlated country: MW
	 with value: 22.0958363301
5th most negatively correlated country: CY
	 with value: 22.8582248059


Some clear trends appear in our regression model. Most obviously, we see that as confirmed above, premium users listen to significantly more music than free users. Gender seems to have little to no correlation with amount of music listened to, and age (measured in years) has a very slight negative correlation. As log account age increases, music listening does too.

Looking at our per-nation trends, we see that Curacao, Russia, Zimbabwe, Puerto Rico, and Nicaragua are the 5 most positively correlated countries. We see that Unknown, India, Iceland, Malawi, and Cyprus are the most negatively correlated countries.

In the following calculations, we assume that Spotify still pays roughly [$0.007 per stream](http://www.theverge.com/2015/12/7/9861372/spotify-year-in-review-artist-payment-royalties).

Our model predicts that a premium user will in the span of half a month, listen to 118 more songs than a premium user. That gives us 2832 more streams in a year, or about \$20 more worth of songs. Since Spotify charges \$60 to \$120 a year for premium, even if we estimate ad revenue per free user per year at half a membership, premium users are still much more lucrative for Spotify.

In the case of our most positively correlated countries, over the course of a year a typical user from Curacao might listen to 12960 more songs, which translates to about \$90 more worth of songs. This seems like a very high amount, and in the case of discounted premium users, Spotify is almost certainly losing money in Curacao.

A check against the user database shows that all of these max/min correlated countries except Iceland only have 1-3 users, so this economic analysis should be performed against a much larger dataset to form any actionable conclusions.

## Concluding remarks

To test whether male and female listeners listen to statistically significant different amounts of music, I chose to perform Welch's t test, as opposed to Student's t test, which assumes equal variances between the populations.

To test whether premium and free users listen to music differently, I performed Welch's t test 7 times, one for each context, because I assumed that the number of songs played between different contexts is independent.

To model usage based on demographic data, because I simultaneously modeled based on nationality and the other demographic data, I did not throw out samples from countries which had few users, even though said samples were likely to be unrepresentative of their countries, because those users still had useful gender, age, and account age data.

Finally, a word on limitations of my exploration and further approaches I would take with more time, data, and experience. Due to how primitive the data captured was (small set of users and short time frame of song data), I found it intractable to explore more interesting questions I had. 

Given a larger dataset over a longer duration of time, I would have liked to model user behavior leading up to the purchase of a subscription. Do listening patterns change significantly in the immediate song plays before a user caves to a premium subscription? If so, perhaps we can then apply this behavioral model to the general population and target more susceptible users to Spotify premium advertisements.

I did not come up a method of segmenting a user's song plays into listening sessions, but if I did that would lend itself very naturally to a measure of which users are the most engaged. If we see a user play entire songs from only one context, say a 500 length playlist, this suggests the user is less engaged and perhaps leaves Spotify on to provide background noise. 

However if a user is actively skipping, around selecting songs from various contexts, then we can say this user is more engaged and is spending more time interacting with the actual UI of Spotify's apps. This user is likely to be more opinionated about the design of the app, and should have their client included in the next round of beta-testing. Also, more engaged users should be worth more to advertisers, which ought to factor into Spotify's advertising fee structure.

Finally, with a much larger dataset of listening habits from around the world, perhaps we see some segment of users are playing much more music than everyone else. Then we might want to audit the individual listening habits of the most prolific users for fraud or other abuse of the system.